In [ ]:
!nvidia-smi

Sun Nov 17 12:44:00 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          On  |   00000000:E1:00.0 Off |                    0 |
| N/A   50C    P0             53W /  300W |       1MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

# 1. Install packages

In [ ]:
!pip install torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118
!pip install mmcv==2.0.1 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0/index.html

!pip install -qq ftfy openmim future tensorboard scikit-learn pandas
!mim install mmengine
!git clone -b main https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .

In [ ]:
# Ensure that you run the notebook from the mmdetection directory
%cd /workspace/mmdetection

/workspace/mmdetection


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# 2. Setup the dataset

In [2]:
# Download dataset from Google Drive
!gdown 16T40TdpaB8VXohm50SySREwrzbuPcJBC
!mkdir -p data/images
!unzip -qqnd data/images images.zip
!mv Train.csv Test.csv SampleSubmission.csv data/

Downloading...
From (original): https://drive.google.com/uc?id=16T40TdpaB8VXohm50SySREwrzbuPcJBC
From (redirected): https://drive.google.com/uc?id=16T40TdpaB8VXohm50SySREwrzbuPcJBC&confirm=t&uuid=b1e406ca-f1fc-458c-b92f-1b1b815bc02c
To: /content/mmdetection/images.zip
100% 4.29G/4.29G [00:56<00:00, 76.5MB/s]


In [3]:
!ls data

images	SampleSubmission.csv  Test.csv	Train.csv


In [6]:
## Download pretrained model weights
!mkdir -p checkpoints
!cd checkpoints && wget -nc https://download.openmmlab.com/mmdetection/v3.0/ddq/ddq_detr_swinl_30e.pth


--2024-11-20 08:17:27--  https://download.openmmlab.com/mmdetection/v3.0/ddq/ddq_detr_swinl_30e.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 8.48.85.230, 8.48.85.226, 8.48.85.228, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|8.48.85.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 883034647 (842M) [application/octet-stream]
Saving to: ‘ddq_detr_swinl_30e.pth’

ddq_detr_swinl_30e. 100%[===================>] 842.13M  31.8MB/s    in 27s     

2024-11-20 08:17:55 (31.0 MB/s) - ‘ddq_detr_swinl_30e.pth’ saved [883034647/883034647]



# 3. Training

In [1]:
import pandas as pd, numpy as np
import sys,os,shutil,gc,re,json,glob,math,time,random,warnings,logging

from tqdm import tqdm
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold,StratifiedGroupKFold
import sklearn.metrics as skm
from sklearn import preprocessing
import torch
from torch import nn
import torch.nn.functional as F
import cv2

from mmengine import Config
from mmengine.runner import Runner
import mmdet

N_SPLITS = 5
RANDOM_STATE = 41
FOLD=0

def fix_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
fix_seed(RANDOM_STATE)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/Users/benx13/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
DIR_DATA = 'data'
CLASSES = {
    'PNN': 0, 'MO': 1, 'MM': 2, 'LyB': 3, 'LGL': 4, 'Thromb': 5, 'LLC': 6, 
    'LAM3': 7, 'EO': 8, 'LY': 9, 'BA': 10, 'MoB': 11, 'LM': 12, 'LH_lyAct': 13,
    'Lysee': 14, 'Er': 15, 'LF': 16, 'LZMG': 17, 'MBL': 18, 'SS': 19, 'PM': 20,
    'B': 21, 'M': 22
}
df = pd.read_csv(f'{DIR_DATA}/train.csv')
df['path'] = f'{DIR_DATA}/images/'+df['NAME']
df['category_id'] = df['class'].map(CLASSES)
df

,NAME,x1,y1,x2,y2,class,path
0,cf7734a2-b.jpg,99,88,266,276,PNN,../data/images/cf7734a2-b.jpg
1,e6a3b579-0.jpg,92,78,272,284,MO,../data/images/e6a3b579-0.jpg
2,60f95776-f.jpg,110,103,251,251,MM,../data/images/60f95776-f.jpg
3,0894e581-c.jpg,112,121,248,262,LyB,../data/images/0894e581-c.jpg
4,3998a979-3.jpg,94,86,276,281,MO,../data/images/3998a979-3.jpg
...,...,...,...,...,...,...,...
52743,ddd3c8e9-5.jpg,133,132,226,231,LF,../data/images/ddd3c8e9-5.jpg
52744,1fa717be-0.jpg,109,112,252,257,M,../data/images/1fa717be-0.jpg
52745,1e923263-9.jpg,174,177,446,429,MoB,../data/images/1e923263-9.jpg
52746,08112a14-8.jpg,122,135,241,248,LyB,../data/images/08112a14-8.jpg


In [5]:
df.describe()

,x1,y1,x2,y2
count,52748.000000,52748.000000,52748.000000,52748.000000
mean,117.975279,117.764162,262.584136,266.793319
std,38.544132,40.123163,54.316188,57.087062
min,-85.000000,-74.000000,13.000000,7.000000
25%,101.000000,101.000000,242.000000,245.000000
50%,114.000000,113.000000,255.000000,259.000000
75%,128.000000,128.000000,267.000000,272.000000
max,572.000000,565.000000,718.000000,812.000000


In [8]:
df.NAME.value_counts()

NAME
0a6762f2-0.jpg    11
8a21dcc3-2.jpg    10
1c82c308-7.jpg    10
74490f37-b.jpg    10
c98091e2-3.jpg     9
                  ..
ffe8d805-7.jpg     1
44cff130-8.jpg     1
5d586687-6.jpg     1
9b6e6e53-5.jpg     1
08112a14-8.jpg     1
Name: count, Length: 48417, dtype: int64

In [ ]:
# df['area'] = (df.xmax-df.xmin)*(df.ymax-df.ymin)
# df.area.describe()

In [9]:
df['class'].value_counts()

class
PNN         6827
Lysee       3408
LAM3        3338
LLC         3158
LyB         3029
MO          2743
MBL         2668
LGL         2439
EO          2432
Thromb      2319
Er          2308
B           2263
LF          2131
LY          2092
M           1994
MM          1707
LH_lyAct    1705
MoB         1531
PM          1152
BA          1001
LM           944
LZMG         845
SS           714
Name: count, dtype: int64

In [10]:
def split_data(df):
    gkf  = StratifiedGroupKFold(n_splits=N_SPLITS,shuffle=True,random_state=RANDOM_STATE)
    df['fold'] = -1
    for fold_id, (train_index, test_index) in enumerate(gkf.split(df,y=df['class'],groups=df.Image_ID)):
        df.loc[test_index,'fold'] = fold_id
    return df

def get_splits(df,fold):
    df_trn = df[df.fold!=fold].copy()
    df_val = df[df.fold==fold].copy()

    return df_trn,df_val

df = split_data(df)
df

,NAME,x1,y1,x2,y2,class,path,fold
0,cf7734a2-b.jpg,99,88,266,276,PNN,../data/images/cf7734a2-b.jpg,1
1,e6a3b579-0.jpg,92,78,272,284,MO,../data/images/e6a3b579-0.jpg,1
2,60f95776-f.jpg,110,103,251,251,MM,../data/images/60f95776-f.jpg,0
3,0894e581-c.jpg,112,121,248,262,LyB,../data/images/0894e581-c.jpg,1
4,3998a979-3.jpg,94,86,276,281,MO,../data/images/3998a979-3.jpg,3
...,...,...,...,...,...,...,...,...
52743,ddd3c8e9-5.jpg,133,132,226,231,LF,../data/images/ddd3c8e9-5.jpg,3
52744,1fa717be-0.jpg,109,112,252,257,M,../data/images/1fa717be-0.jpg,4
52745,1e923263-9.jpg,174,177,446,429,MoB,../data/images/1e923263-9.jpg,2
52746,08112a14-8.jpg,122,135,241,248,LyB,../data/images/08112a14-8.jpg,1


In [11]:

df['split'] = 'train'
df.loc[df.fold==FOLD,'split'] = 'val'
df.split.value_counts()



split
train    42133
val      10615
Name: count, dtype: int64

In [18]:


def gen_anno(meta,split):
    annotations = []
    images = []
    anno_id = 0
    image_id = 0

    meta=meta.copy()
    for file_name,d in tqdm(meta.groupby('NAME')):
        path = d.path.values[0]
        im = cv2.imread(path)
        height,width,_ = im.shape
        for _,row in d.iterrows():
            x0, y0, x1, y1 = row.x1, row.y1, row.x2, row.y2
            w = x1 - x0
            h = y1 - y0
            bbox = np.array([x0, y0, w, h]).tolist()
            area = w * h
            if area<1:
                #print('skipping',file_name,area)
                continue

            anno = dict(
                        image_id = image_id,
                        id = anno_id,
                        category_id = row.category_id,
                        bbox = bbox,
                        area = area,
                        iscrowd = 0
                    )
            anno_id += 1
            annotations.append(anno)

        images.append(dict(id=image_id, file_name=file_name,height=height,width=width))
        image_id += 1

    categories = [dict(id=id, name=name) for name, id in CLASSES.items()]
    coco_json = dict(images=images, annotations=annotations, categories=categories)
    path = f'data/{split}.json'
    print(f'written {len(annotations)} annotations for {len(images)} images to {path}')
    with open(path,'w', encoding='utf-8') as f:
        json.dump(coco_json,f,ensure_ascii=False)



In [19]:

gen_anno(df[df.split=='val'],'val')
### gen_anno(df[df.split=='train'],'train')
# training on complete dataset
gen_anno(df,'train')


100%|██████████| 9685/9685 [00:14<00:00, 657.93it/s]


written 10615 annotations for 9685 images to ../data/val.json


100%|██████████| 48417/48417 [01:15<00:00, 645.06it/s]


written 52748 annotations for 48417 images to ../data/train.json


In [20]:
!ls ../data

Test.csv   Train.csv  images     train.json val.json


In [ ]:
cfg_path = 'configs/ddq/ddq-detr-4scale_swinl_8xb2-30e_coco.py'
cfg = Config.fromfile(cfg_path)

load_from = 'checkpoints/ddq_detr_swinl_30e.pth'

data_dir = 'data'
img_prefix = 'images'
max_epochs = 30
val_interval = 1

bs = 2
cfg.train_num_workers = num_workers = 8
cfg.train_dataloader.batch_size = bs

cfg.data_root = data_dir
cfg.work_dir = './output'
metainfo = {
    'classes': tuple(CLASSES.keys()),
    'palette': [
        (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) 
        for _ in range(len(CLASSES))
    ]
}
cfg.dataset_type = 'CocoDataset'

num_classes = 23

cfg.num_classes = num_classes
cfg.model.bbox_head.num_classes = num_classes

cfg.train_pipeline =[
    dict(backend_args=None, type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(prob=0.5, type='RandomFlip'),

    dict(
        transforms=[
            [
                dict(
                    keep_ratio=True,
                    scales=[
                        (384, 384),
                        (416, 416),
                        (448, 448),
                        (480, 480),
                        (512, 512),
                    ],
                    type='RandomChoiceResize'),
            ],
            [
                dict(
                    keep_ratio=True,
                    scales=[(352, 352), (384, 384), (416, 416)],
                    type='RandomChoiceResize'),
                dict(
                    allow_negative_crop=True,
                    crop_size=(320, 320),
                    crop_type='absolute_range',
                    type='RandomCrop'),
                dict(
                    keep_ratio=True,
                    scales=[(384, 384), (416, 416), (448, 448)],
                    type='RandomChoiceResize'),
            ],
        ],
        type='RandomChoice'),

    dict(type='YOLOXHSVRandomAug'),
    dict(type='Sharpness',prob=0.5),
    dict(type='AutoContrast',prob=0.5,min_mag=0.1,max_mag=1.9,level=10),
    dict(type='Rotate', level=10, min_mag=180.,max_mag=180.,prob=0.5),

    dict(type='PackDetInputs'),
    ]


cfg.train_dataloader.dataset=dict(
        data_root=cfg.data_root,
        metainfo=metainfo,
        ann_file='train.json',
        backend_args=None,
        data_prefix=dict(img=img_prefix),
        filter_cfg=dict(filter_empty_gt=False),
        pipeline=cfg.train_pipeline,

        type='CocoDataset')

cfg.train_dataloader.num_workers = cfg.val_dataloader.num_workers = num_workers
cfg.val_dataloader.dataset=dict(
        data_root=cfg.data_root,
        metainfo=metainfo,
        data_prefix=dict(img=img_prefix),
        ann_file='val.json',
        pipeline = cfg.test_pipeline,
        test_mode=True,
    type='CocoDataset')

cfg.test_dataloader = cfg.val_dataloader

cfg.load_from = load_from
cfg.train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=max_epochs, val_interval=val_interval)
cfg.visualizer = dict( name='visualizer',type='DetLocalVisualizer',vis_backends=[dict(type='LocalVisBackend'),dict(type='TensorboardVisBackend')])
cfg.val_evaluator = cfg.test_evaluator = dict(
    ann_file='data/val.json',
    backend_args=None,
    format_only=False,
    metric=[
        'bbox',
    ],
    type='CocoMetric')

In [ ]:
fix_seed(RANDOM_STATE)
runner = Runner.from_cfg(cfg)
runner.train()

In [ ]:
# !cp output/epoch_30.pth ./